# JellyFish

**CONTENTS**:

+ **Introduction** - Outlining the Problem
+ **JellyFish and It's Algorithms** - Brief introduction to JellyFish
+ **Levenshtein Distance** - Description, Calculation, and Examples
+ **Damerau Levenshtein Distance** - Description, Calculation, and Examples
+ **Jaro Distance** - Description, Calculation, and Examples
+ **Jaro-Winkler Distance** - Description, Calculation, and Examples

In [1]:
import pandas as pd
import numpy as np
import datetime
import os
from pandas import DataFrame
from numpy import nan as NA
from IPython.core.display import HTML
from IPython.core.display import Image
from IPython.display import Math
from IPython.display import Latex
import collections
import jellyfish as jf
import re

ImportError: No module named pandas

## Introduction

With regard to style names in production data we face two distinct problems:

1. Style names not being clean within factory. <br />
</p> </p> The fact of the matter is that often the string values of the style names are not clean meaning that styles that in reality are the same are not recognised as such. This is clearly problematic as we want to be able to ;identify those styles that are produced on multiple lines in order to sensibly create the running days values, &nbsp;&nbsp;&nbsp;to do across line analysis of the same styles etc. etc. 

2. Matching styles across factory. <br />
</p> </p> Eventually we will want to be able to match styles across factories in order to identify those styles that are produced in multiple factories. The reasons for this are pretty obvious. 


## JellyFish and Its Algorithms

Jellyfish is a python library for doing approximate and phonetic matching of strings. It allows for string comparison using the following algorithms:

+ Levenshtein Distance
+ Damerau-Levenshtein Distance
+ Jaro Distance
+ Jaro-Winkler Distance
+ Match Rating Approach Comparison
+ Hamming Distance

All of the above are metrics for measuring the difference between two sequences. Each of these will not be briefly introduced and an example given:


<br />

###**Levenshtein Distance**<br />
The Levenshtein distance between two words is the minimum number of single-character edits (insertion, deletion, substitution) required to transform one string into another. The lower the score, the lower the number of edits. If the score is 0, then no edits are needed so the strings are exactly the same. 

For example, the Levenshtein distance between "kitten" and "sitting" is 3, since the following three edits change one into the other, and there is no way to do it with fewer than three edits:

+ Edit 1: kitten → sitten (substitution of "s" for "k")
+ Edit 2: sitten → sittin (substitution of "i" for "e")
+ Edit 3: sittin → sitting (insertion of "g" at the end)

The measure is most useful when looking to match short strings with longer strings. It takes a lot of computational power to do this with long strings and may not therefore be totally appropriate.  

For more information about the measure see

http://en.wikipedia.org/wiki/Levenshtein_distance

Now some examples follow:

In [4]:
A = 'Rory'
B = 'Ronnie'

jf.levenshtein_distance(unicode(A), unicode(B))

4

As the following demonstrates, the algorithm also considers case, which is a strong argument for converting all strings to the same case. This is a pretty standard way of working with strings so will come as no surprise.

In [6]:
A = 'Rory'
B = 'rory'
jf.levenshtein_distance(unicode(A), unicode(B))

1

In [8]:
# Now the levenshtein score of 0 means the strings are an exact match. 
jf.levenshtein_distance(unicode(A.lower()), unicode(B.lower()))

0


<br />

###**Damerau Levenshtein Distance**<br />
This measure is very similar to the Levenshtein distance, in that it is a measure of the minimum number of edits needed to transform one string into another. The permissible 'edits' in the Levenshtein distance are insertions, deletion and substitution whereas in the Damerau Levenshtein distance the transposition of two adjacent characters is also allowed. Damerau claimed that these four edits correspond to 80% of human spelling errors. 

As with the Levenshtein distance a score of zero indicates and exact match etc.

This measure may be an improvement on the Levenshtein distance as using the Damerau Levenshtein Distance strings where two letters are simply the wrong way around will have a lower score (indicating a better match) than they would under the Levenshtein measure. 

A simple example will suffice:


In [12]:
jf.levenshtein_distance(unicode('jellyfihs'), unicode('jellyfish'))

2

In [13]:
jf.damerau_levenshtein_distance(unicode('jellyfihs'), unicode('jellyfish'))

1

In this example, the **Levenshtein** distance works like this:

+ Edit 1: jellyfihs → jellyfiss (substitution of "s" for "h")
+ Edit 2: jellyfiss → jellyfish (substitution of "h" for "s")

This is only one way it could happen. For instance 'h' could have been deleted, and then inserted (and so on), but the minimum number of edits is always 2

The **Damerau Levenshtein** distance works like this:

+ Edit 1: jellyfihs → jellyfish (transposition of 's' and 'h' adjacent characters)

The measure may therefore be a better one in that it recognises strings as being closer than the Levenshtein measure in cases where there has been a simple mix up of characters. 


**NB** I have observed some odd behaviour of the jf.damerau_levenshtein_distance algorithm. The odd behaviour may be related to me misunderstanding the nature of the measure, or it may be a problem with coding of the library. I have written to the developer for clarification. If you are interested, then see the following, if not then skip to the next measure below.


In [15]:
jf.damerau_levenshtein_distance(unicode('jellyfihs'), unicode('jellyfish'))

1

In [17]:
jf.damerau_levenshtein_distance(unicode('ifhs'), unicode('fish'))

2

I find the above output very odd for the reason that the scores should be the same as the edits required in both instances are exactly the same:

In the first example:

+ Edit 1: jellyifhs → jellyfihs (transpose adjacent characters 'i' and 'f')
+ Edit 2: jellyfihs → jellyfish (transpose adjacent characters 'h' and 's')

In the second example:

+ Edit 1: ifhs → fihs (transpose adjacent characters 'i' and 'f')
+ Edit 2: fihs → fish (transpose adjacent characters 'h' and 's')

Why the outputs are different remains to be determined.

**Update** It appears from looking at the source code that in some cases the function is returning the OSA  measure not the Damerau Levenshtein measure. The developer is now aware, and has been contacted. use this measure with caution. 

More information on the measure can be found at http://en.wikipedia.org/wiki/Damerau%E2%80%93Levenshtein_distance

<br />

###**Jaro Distance**<br />
The Jaro distance is a measure that considers the number of matching characters in both strings being compared, and also the number of transpositions which is defined as the number of matching characters (in a different sequence order) divided by two. The measure returns a score between 0 and 1, 0 being no match whatsoever (as defined in the calculation) and 1 being a perfect match. 

Beware that this measure will ignore matching characters that are more than a certain distance from each other. This could either be a good thing (to ignore spurious matches) or a bad thing (ignoring correct matches). In any event, it is important to be aware of this, so it is explained in detail below.

It is calculated as follows:


In [18]:
Latex(r"""\begin{eqnarray}
d_j = \left\{
    \begin{array}{1 1}
        0 & \quad \text{if $m$ = 0 <}\\
        \\
        \frac{1}{3} \bigg(\frac{m}{|s_1|} + \frac{m}{|s_2|} + \frac{m - t}{m} & \quad \text{otherwise}
    \end{array} \right.
\end{eqnarray}""")

<IPython.core.display.Latex object>

+ **$m$** is the number of matching characters
+ **$t$** is half the number of transpositions.
+ **$|s_1|$** is the length of the first string to be matched
+ **$|s_2|$** is the length of the second string to be matched

If the number of matching characters is 0, then the measure equals 0

A character in each string is only considered to be matching if it is the same and obeys the following rule as to distance:

In [19]:
Latex(r"""\begin{eqnarray}
\frac{max(|s_1|,|s_2|)}{2} -1
\end{eqnarray}""")

<IPython.core.display.Latex object>

Observe the following:


In [21]:
S1 = 'AzzzzB'
S2 = 'BxxA'
jf.jaro_winkler(unicode(S1), unicode(S2))

0

Although the characters A and B appear in both strings, m = 0 because they are farther in distance from each other than:

In [22]:
Latex(r"""\begin{eqnarray}
\frac{max(|6|,|4|)}{2} -1 = 2
\end{eqnarray}""")

<IPython.core.display.Latex object>

Transpositions are also important as already mentioned. The number of transpositions is calculated as "The number of matching (but different sequence order) characters divided by 2". Observe the following:

In [24]:
S3 = 'Poverty'
S4 = 'Poervty'
jf.jaro_distance(unicode(S3), unicode(S4))

0.9523809523809524

Sadly it looks like this is also being calculated incorrectly. To my mind the calculation should be as follows:

In [25]:
Latex(r"""\begin{eqnarray}
\Bigg(\frac{7}{7} + \frac{7}{7} + \frac{7- \frac{3}{2}}{7}\Bigg) = 0.9285714
\end{eqnarray}""")

<IPython.core.display.Latex object>

Whereas is appears that it is being calcualted as follows:

In [26]:
Latex(r"""\begin{eqnarray}
\Bigg(\frac{7}{7} + \frac{7}{7} + \frac{7- \frac{2}{2}}{7}\Bigg) = 0.9523809
\end{eqnarray}""")

<IPython.core.display.Latex object>

The critical difference is that I calculate the number of matching (but different sequence order) characters as 3, those being:

+ e
+ r
+ v

Whereas it appears that JellyFish thinks there are only two.

Again, I have raised this issue with the developer. It may resolved in future, but for now use this measure with caution. 

<br />

###**Jaro-Winkler Distance**
The Jaro-Winkler Distance measure builds upon the Jaro measure, but uses a prefix scale which gives more favorable ratings to strings that match from the beginning for a set prefix length. This will become clear when the calculation is viewed:

In [27]:
Latex(r"""\begin{eqnarray}
d_w = d_j + (\ell p(1 - d_j))
\end{eqnarray}""")

<IPython.core.display.Latex object>

The expression is made up of the following:

+ **$d_w$** is the Jarrow-Winkler distance
+ **$d_j$** is the Jaro distance for strings s1 and s2
+ **$\ell$** is the length of the common prefix up to a maximum of 4 characters
+ **$p$** is a constant scaling factor for how much the score is adjusted upwards for having common prefixes. It should not exceed  0.25, otherwise the distance can become larger than 1. The standard value for this constant in Winkler's work is 0.1
    
    

The key takeaway is that strings that begin with the same prefixes score more highly...

Observe the following:

In [29]:
S5 = 'Innovaiosn'
S6 = 'Innovations'
jf.jaro_distance(unicode(S5), unicode(S6))

0.9363636363636364

In [30]:
jf.jaro_winkler(unicode(S5), unicode(S6))

0.9618181818181818

Although it is not stated anywhere in the jellyfish documentation, it is clear that the value of $p$ is 0.1, this is because rearranging the following to solve for $\ell$:

In [31]:
Latex(r"""\begin{eqnarray}
d_w = d_j + (\ell p(1 - d_j))\\
\\
\\
\ell = \frac{d_w - d_j}{1 - d_j} * \frac{1}{4}\\
\\
\\
0.1 = \frac{0.96182-0.936364}{1-0.936364} * \frac{1}{4}

\end{eqnarray}""")

<IPython.core.display.Latex object>

In some implementations of Jaro-Winkler, the prefix bonus is only added when the compared strings have a Jaro distance above a set "boost threshold". In the work of the developer himself, this threshold was set at 0.7. In other words, if the Jaro measure is less than 0.7, then even if the prefixes of the string are the same up to four characters, the prefix bonus will not be applied. Then the calculation looks like this:

In [32]:
Latex(r"""\begin{eqnarray}
d_w = \left\{
    \begin{array}{1 1}
        d_j & \quad \text{if $d_j$ < $b_t$}\\
        d_j + (\ell p(1 - d_j)) & \quad \text{otherwise}
    \end{array} \right.
\end{eqnarray}""")

<IPython.core.display.Latex object>

Again, as there is no documentation to JellyFish it is hard to know if this is being applied, and if so, at what level the $b_t$ value is set to trigger the prefix bonus. However, a bit of easy experimentation can demonstrate, firstly I compare strings that have a Jaro score just below 0.7, and then strings that have a score just above 0.7

In [34]:
S7 = 'ABCDqwerty'
S8 = 'ABCDpoiuyt'
jf.jaro_distance(unicode(S7), unicode(S8))

0.6777777777777777

In [35]:
jf.jaro_winkler(unicode(S7), unicode(S8))

0.6777777777777777

In [36]:
S9 = 'ABCDqwerty'
S10 = 'ABCDpoiuty'
jf.jaro_distance(unicode(S9), unicode(S10))

0.7333333333333334

In [37]:
jf.jaro_winkler(unicode(S9), unicode(S10))

0.8400000000000001

The above output indicates that indeed, there implementation of the Jaro-Winkler measure in JellyFish uses a threshold of 0.7 for $b_t$

Developer - Pranav Shukla <br>
Email - pranavdynamic@gmail.com